# Fonte de Alimentação Estabilizada

[Ver mais](https://www.notion.so/guilhermebaos/18-Circuitos-com-D-odos-bec037cdcecb4f36a6c0c4d5c225115c?pvs=4#1d22979f33d44f5b87287e51af381d29) - Teoria!

In [2]:
import csv
import numpy as np

# Parâmetros
VP = 20.747
F = 50

RR = np.array([150, 220, 470, 1000, 1500, 2200, 4700, 10000, 1e06])
CC = np.array([100e-06, 1000e-06])


# Tensão nas placas do condensador caso este descarregue alpha * Tau
def Vc(alpha: float, VMax: float, ftau: float) -> float:
    return VMax * np.exp(-alpha/ftau)


# Tensão à saída do transformador
def Vt(alpha: float) -> float:
    global VP

    return VP * np.cos(2 * np.pi * alpha)


# Determinar o tempo que o condensador está a descarregar
N = 10001

aa = np.linspace(0, 1, N)
cc = Vt(aa)

valores = []
for c in CC:
    for r in RR:
        ftau = F * r * c

        # O condensador começa a descarregar quando conseguir manter uma tensão superior à tensão de entrada
        for index, alpha in enumerate(aa):
            step = aa[index+1] - alpha

            Vt_1 = Vt(alpha)
            Vt_2 = Vt(aa[index+1])

            Vc_1 = Vc(0, Vt_1, ftau)
            Vc_2 = Vc(step, Vt_1, ftau)

            if Vc_2 > Vt_2:
                break
        alpha0 = alpha

        # Carga do condensador quando começa a descarregar
        VMax = Vt(alpha0)
        ee = Vc((aa-alpha0), VMax, ftau)
        dd = cc - ee

        # O condensador carrega se cc >= ee
        carga = aa[dd >= 0]

        # O tempo de descarga é o tempo entre os momentos em que está a carregar
        diffcarga = np.diff(carga)
        descarga = np.max(diffcarga)

        valores += [(c, r, VP - Vc(descarga, VMax, ftau))]
        print(valores[-1])


titulos = ["C/ F", "R/ Ohm", "V_ripple/ V"]
with open("01-Dados.csv", "w+", encoding="UTF-8", newline="") as ficheiro:
    writer = csv.writer(ficheiro)

    writer.writerow(titulos)
    for item in valores:
        writer.writerow(item)
        

(0.0001, 150.0, 13.51136502872113)
(0.0001, 220.0, 10.879100335664404)
(0.0001, 470.0, 6.396286889297427)
(0.0001, 1000.0, 3.43381840639368)
(0.0001, 1500.0, 2.397785103887127)
(0.0001, 2200.0, 1.6893239792740111)
(0.0001, 4700.0, 0.825764289013744)
(0.0001, 10000.0, 0.398021316204634)
(0.0001, 1000000.0, 0.004136124516044504)
(0.001, 150.0, 2.397785103887127)
(0.001, 220.0, 1.6893239792740111)
(0.001, 470.0, 0.825764289013744)
(0.001, 1000.0, 0.398021316204634)
(0.001, 1500.0, 0.2677833885056806)
(0.001, 2200.0, 0.18378920193548964)
(0.001, 4700.0, 0.08681808091501608)
(0.001, 10000.0, 0.04103836455931997)
(0.001, 1000000.0, 0.0004145209189232446)
